### PREPARATION


In [1]:
!pip install pulp

import math
import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
import pandas as pd
import pulp
import itertools
import requests
import random
from geopy.distance import geodesic
import time
import pandas as pd
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 22.0 MB/s eta 0:00:00


In [2]:
def generate_list(n, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")
    result = [0]
    random_values = np.random.rand(n - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    result.extend(rounded_values)

    return result

def generate_demand_list(n, m, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if m >= n:
        raise ValueError("M must be less than N.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")

    random_values = np.random.rand(n - m - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    zeros_list = [0] * m

    result = list(rounded_values) + zeros_list

    random.shuffle(result)

    result = [-1] + result

    return result

def replace_zeros_with_sum(input_list, target_sum):
    zero_positions = [i for i, value in enumerate(input_list) if value == 0]
    num_zeros = len(zero_positions)

    result = input_list.copy()

    if num_zeros == 0:
        raise ValueError("There are no zeros to replace in the list.")

    random_values = np.random.rand(num_zeros)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    for i, pos in enumerate(zero_positions):
        result[pos] = rounded_values[i]

    return result

def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append("->".join(map(str, current_path)))
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths
def repetitions(list1, list2):
  common_elements = set(list1) & set(list2)
  return (2*len(common_elements))/len(list1+list2)

def equal_paths(list1, list2):
  are_equal = sorted(list1) == sorted(list2)
  if are_equal:
    return 1
  else:
    return 0

In [3]:
data = {
    'Original Latitude': [50.050360, 50.073607, 50.094664, 50.059735, 50.041098,
                          50.095299, 50.011595, 50.040565, 50.036358, 50.007445, 50.000989],
    'Original Longitude': [19.963734, 20.045935, 20.017257, 19.867351, 20.000920,
                           19.944809, 20.034836, 20.028741, 19.937026, 20.002817, 19.998498],
    'Snapped Latitude': [50.050428, 50.073384, 50.094609, 50.059320, 50.041162,
                         50.095389, 50.011603, 50.040220, 50.036059, 50.007440, 50.001006],
    'Snapped Longitude': [19.963786, 20.045741, 20.016973, 19.867208, 20.000314,
                          19.944873, 20.034905, 20.028809, 19.936960, 20.002814, 19.998493]
}

df = pd.DataFrame(data)

print(df)
number_of_clients = 100
cost_R = np.ones((number_of_clients + 1, number_of_clients + 1))
cost_T = np.ones((number_of_clients + 1, number_of_clients + 1))
cost_G = np.ones((number_of_clients + 1, number_of_clients + 1))

    Original Latitude  Original Longitude  Snapped Latitude  Snapped Longitude
0           50.050360           19.963734         50.050428          19.963786
1           50.073607           20.045935         50.073384          20.045741
2           50.094664           20.017257         50.094609          20.016973
3           50.059735           19.867351         50.059320          19.867208
4           50.041098           20.000920         50.041162          20.000314
5           50.095299           19.944809         50.095389          19.944873
6           50.011595           20.034836         50.011603          20.034905
7           50.040565           20.028741         50.040220          20.028809
8           50.036358           19.937026         50.036059          19.936960
9           50.007445           20.002817         50.007440          20.002814
10          50.000989           19.998498         50.001006          19.998493


### CVRP

In [4]:
def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append("->".join(map(str, current_path)))
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths

def create_path_as_list(pairs_list):
    wanted_length = len(pairs_list)
    path = ''
    current_pair = next(pair for pair in pairs_list if pair[0] == 0)
    path += str(current_pair[0])

    next_start = current_pair[1]

    pairs_list.remove(current_pair)

    while len(path) < wanted_length:
        path += str(next_start)
        current_pair = next(pair for pair in pairs_list if pair[0] == next_start)
        pairs_list.remove(current_pair)
        next_start = current_pair[1]

    return path

def cvrp(number_of_clients, number_of_vehicles, cost_buffor, demand_of_client):
    problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

    capacity_of_vehicle = 100
    x = [[[pulp.LpVariable(f"x{i}_{j},{k}", cat="Binary") if i != j else None
          for k in range(number_of_vehicles)]
          for j in range(number_of_clients + 1)]
        for i in range(number_of_clients + 1)]

    problem += pulp.lpSum(cost_buffor[i][j] * x[i][j][k] if i != j else 0
                          for k in range(number_of_vehicles)
                          for j in range(number_of_clients + 1)
                          for i in range(number_of_clients + 1))

    for j in range(1, number_of_clients + 1):
        problem += pulp.lpSum(x[i][j][k] if i != j else 0
                              for i in range(number_of_clients + 1)
                              for k in range(number_of_vehicles)) == 1

    for k in range(number_of_vehicles):
        problem += pulp.lpSum(x[0][j][k] for j in range(1, number_of_clients + 1)) == 1
        problem += pulp.lpSum(x[i][0][k] for i in range(1, number_of_clients + 1)) == 1

    for k in range(number_of_vehicles):
        for j in range(number_of_clients + 1):
            problem += pulp.lpSum(x[i][j][k] if i != j else 0
                                  for i in range(number_of_clients + 1)) - \
                      pulp.lpSum(x[j][i][k] for i in range(number_of_clients + 1)) == 0

    for k in range(number_of_vehicles):
        problem += pulp.lpSum(demand_of_client[j] * x[i][j][k] if i != j else 0
                              for i in range(number_of_clients + 1)
                              for j in range(1, number_of_clients + 1)) <= capacity_of_vehicle

    u = pulp.LpVariable.dicts("u", range(1, number_of_clients + 1), 0, number_of_clients, cat="Continuous")
    for i in range(1, number_of_clients + 1):
        for j in range(1, number_of_clients + 1):
            if i != j:
                for k in range(number_of_vehicles):
                    problem += u[i] - u[j] + (number_of_clients * x[i][j][k]) <= number_of_clients - 1

    problem.solve()

    all_uesd_edges = []
    paths = [[] for _ in range(number_of_vehicles)]

    for k in range(number_of_vehicles):
        for i in range(number_of_clients + 1):
            for j in range(number_of_clients + 1):
                if i != j and pulp.value(x[i][j][k]) == 1:
                    all_uesd_edges.append((i, j))
                    paths[k].append((i, j))
    print(all_uesd_edges)

    result_paths = find_paths_vrb(all_uesd_edges, number_of_vehicles)
    print(result_paths)

    return pulp.value(problem.objective)

In [5]:
number_of_clients = 10
number_of_vehicles = 2
capacity_of_vehicle = 100
demand_of_client = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)

In [6]:
demand_of_client = generate_list(number_of_clients + 1, 0.85 * capacity_of_vehicle * number_of_vehicles)
cvrp(number_of_clients, number_of_vehicles, cost_R, demand_of_client)

[(0, 4), (2, 0), (3, 6), (4, 3), (5, 2), (6, 5), (0, 10), (1, 8), (7, 1), (8, 0), (9, 7), (10, 9)]
['0->4->3->6->5->2->0', '0->10->9->7->1->8->0']


12.0

In [7]:
demand_of_client = generate_list(number_of_clients + 1, 0.85 * capacity_of_vehicle * number_of_vehicles)
cvrp(number_of_clients, number_of_vehicles, cost_G, demand_of_client)

[(0, 9), (4, 8), (8, 10), (9, 4), (10, 0), (0, 5), (1, 0), (2, 6), (3, 2), (5, 7), (6, 1), (7, 3)]
['0->9->4->8->10->0', '0->5->7->3->2->6->1->0']


12.0

### VRPTW


In [8]:
def find_paths_vrb(edges, k, depot_one_more_time):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == depot_one_more_time and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths

def vrptw(number_of_clients, number_of_vehicles, cost_buffor, a, b, t, T):
      n = number_of_clients
      c = number_of_vehicles
      Q = 100
      M = 10000000


      cost_shiffted = cost_buffor.tolist()
      cost_shiffted.append(cost_shiffted[0])
      iterator = 0
      for row in cost_shiffted:
        cost_shiffted[iterator].append(cost_shiffted[iterator][0])
        iterator += 1

      T = T.tolist()
      T.append(T[0])
      iterator = 0
      for row in T:
        T[iterator].append(T[iterator][0])
        iterator += 1

      prob = pulp.LpProblem("VRPTW", pulp.LpMinimize)

      x = pulp.LpVariable.dicts("x", [(i,j,k) for i in range(n+1) for j in range(1, n+2) for k in range(c)], cat='Binary')
      s = pulp.LpVariable.dicts("s", [(i,k) for i in range(n+2) for k in range(c)], lowBound=0, cat='Continuous')

      prob += pulp.lpSum(cost_shiffted[i][j] * x[(i,j,k)] for i in range(n+1) for j in range(1, n+2) for k in range(c))

      for j in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for i in range(n+1) for k in range(c)) == 1

      for i in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for j in range(1, n+2) for k in range(c)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[i,n+1,k] for i in range(n+1)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[0,j,k] for j in range(1, n+2)) == 1

      for m in range(1, n+1):
        for k in range(c):
          prob += pulp.lpSum(x[(i,m,k)] for i in range(n+1) if i != m) == pulp.lpSum(x[(m,j,k)] for j in range(1,n+2) if j != m)

      for k in range(c):
        prob += x[0,n+1,k] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            if i == j:
              prob += x[(i,j,k)] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            prob += s[(i,k)] + t[i] + T[i][j] <= s[(j,k)] + M*(1 - x[(i,j,k)])

      for i in range(1, n+1):
        for k in range(c):
          prob += M*(pulp.lpSum(x[i,j,k] for j in range(1,n+2)) - 1) + a[i] <= s[(i,k)]

      for i in range(1, n+1):
        for k in range(c):
          prob += s[(i,k)] + t[i] <= b[i] + M*(1 - pulp.lpSum(x[(i,j,k)] for j in range(1,n+2)))


      prob.solve()
      print(pulp.LpStatus[prob.status])

      all_uesd_edges = []
      paths = [[] for _ in range(c)]

      for k in range(c):
          for i in range(n + 1):
              for j in range(1, n + 2):
                  if i != j and pulp.value(x[i,j,k]) == 1:
                      all_uesd_edges.append((i, j))
                      paths[k].append((i, j))
      print(all_uesd_edges)
      result_paths = find_paths_vrb(all_uesd_edges, number_of_clients, number_of_clients+1)

      print(result_paths)

In [9]:
number_of_clients = 10
capacity_of_vehicle = 100
number_of_vehicles = 2
service_time = 5

median = np.median(cost_T)
max_time = math.floor(number_of_clients * median + service_time*number_of_clients/number_of_vehicles) +1

t = [service_time] * (number_of_clients + 10)
a = [0] * (number_of_clients + 1)
b = [max_time] * (number_of_clients + 2)

a = [0, 60, 30, 0, 150, 0, 60, 40, 0, 0, 150]
b = [186, 100, 50, 186, 150, 186, 186, 100, 60, 186, 186, 186]

time_window_time = 40

for client in range(number_of_clients + 1):
    a[client] = random.randint(int(median/2), int(max_time - service_time - median))
    b[client] = a[client] + time_window_time

a[0] = 0
b[0] = max_time
print(a[1:])
print(b[1:])

[20, 5, 22, 26, 22, 27, 17, 4, 8, 14]
[60, 45, 62, 66, 62, 67, 57, 44, 48, 54, 186]


In [10]:
vrptw(number_of_clients, number_of_vehicles, cost_R, a, b, t, cost_T)

Optimal
[(0, 2), (2, 8), (3, 10), (5, 3), (6, 5), (8, 6), (10, 11), (0, 1), (1, 7), (4, 9), (7, 4), (9, 11)]
[[0, 2, 8, 6, 5, 3, 10, 11], [0, 1, 7, 4, 9, 11]]


In [11]:
vrptw(number_of_clients, number_of_vehicles, cost_G, a, b, t, cost_T)

Optimal
[(0, 2), (2, 8), (3, 10), (5, 3), (6, 5), (8, 6), (10, 11), (0, 1), (1, 7), (4, 9), (7, 4), (9, 11)]
[[0, 2, 8, 6, 5, 3, 10, 11], [0, 1, 7, 4, 9, 11]]


### VRPSDP

In [12]:
demand_of_client = generate_list(number_of_clients + 1, 0.8* capacity_of_vehicle * number_of_vehicles)

In [13]:
def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append("->".join(map(str, current_path)))
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths


def vrpsdp(number_of_clients, number_of_vehicles, cost_buffor,p,d):
  capacity_of_vehicle = 100
  vrpsdp = pulp.LpProblem("VRPSDP", pulp.LpMinimize)
  m = number_of_vehicles
  x = pulp.LpVariable.dicts("x", [(i, j, k) for i in range(number_of_clients+1) for j in range(number_of_clients+1) for k in range(m)], cat='Binary')
  y = pulp.LpVariable.dicts("y", [(i, j) for i in range(number_of_clients+1) for j in range(number_of_clients+1)], lowBound=0, cat='Continuous')
  z = pulp.LpVariable.dicts("z", [(i, j) for i in range(number_of_clients+1) for j in range(number_of_clients+1)], lowBound=0, cat='Continuous')

  vrpsdp += pulp.lpSum(cost_buffor[i][j] * x[i, j, k] for k in range(m) for i in range(number_of_clients+1) for j in range(number_of_clients+1) if i != j)

  Q = capacity_of_vehicle

  for j in range(1, number_of_clients+1):
      vrpsdp += pulp.lpSum(x[i, j, k] for k in range(m) for i in range(number_of_clients+1) if i != j) == 1

  for k in range(m):
    for j in range(number_of_clients+1):
      vrpsdp += pulp.lpSum(x[i,j,k] for i in range(number_of_clients+1)) - pulp.lpSum(x[j,i,k] for i in range(number_of_clients+1)) == 0

  for k in range(m):
    vrpsdp += pulp.lpSum(x[0,j,k] for j in range(1, number_of_clients+1)) == 1

  for j in range(1, number_of_clients+1):
    vrpsdp += pulp.lpSum(y[j,i] for i in range(number_of_clients+1)) - pulp.lpSum(y[i,j] for i in range(number_of_clients+1)) == p[j]

  for j in range(1, number_of_clients+1):
    vrpsdp += pulp.lpSum(z[i,j] for i in range(number_of_clients+1)) - pulp.lpSum(z[j,i] for i in range(number_of_clients+1)) == d[j]

  for i in range(number_of_clients+1):
    for j in range(number_of_clients+1):
      if i != j:
        vrpsdp += y[i,j] + z[i,j] <= Q * pulp.lpSum(x[i,j,k] for k in range(m))

  vrpsdp += pulp.lpSum(x[0,j,k] for j in range(1, number_of_clients+1) for k in range(m)) == m
  for i in range(1,number_of_clients+1):
    vrpsdp += pulp.lpSum(x[i,j,k] for j in range(number_of_clients+1) for k in range(m) if i != j) == 1

  vrpsdp.solve()

  print(pulp.value(vrpsdp.objective))

  all_uesd_edges_road = []
  paths = [[] for _ in range(number_of_vehicles)]

  for k in range(number_of_vehicles):
      for i in range(number_of_clients + 1):
          for j in range(number_of_clients + 1):
              if i != j and pulp.value(x[i,j,k]) == 1:
                  all_uesd_edges_road.append((i, j))
                  paths[k].append((i, j))
  result = find_paths_vrb(all_uesd_edges_road, number_of_vehicles)
  print(all_uesd_edges_road)

  print(result)

In [14]:
number_of_clients = 10
capacity_of_vehicle = 100
number_of_vehicles = 2

p = generate_list(number_of_clients + 1, 0.85 * capacity_of_vehicle * number_of_vehicles)
d = generate_list(number_of_clients + 1, 0.85 * capacity_of_vehicle * number_of_vehicles)

In [15]:
print(d[1:])
print(p[1:])

[19.01, 5.2, 25.16, 4.15, 21.59, 28.0, 23.82, 23.7, 8.72, 10.65]
[23.86, 7.96, 5.69, 21.69, 27.98, 14.59, 19.31, 27.56, 17.46, 3.9000000000000092]


In [16]:
vrpsdp(number_of_clients, number_of_vehicles, cost_R ,p,d)

12.0
[(0, 8), (3, 4), (4, 9), (6, 3), (8, 6), (9, 0), (0, 2), (1, 5), (2, 7), (5, 10), (7, 1), (10, 0)]
['0->8->6->3->4->9->0', '0->2->7->1->5->10->0']


In [17]:
p = generate_list(number_of_clients + 1, 0.85 * capacity_of_vehicle * number_of_vehicles)
d = generate_list(number_of_clients + 1, 0.85 * capacity_of_vehicle * number_of_vehicles)

vrpsdp(number_of_clients, number_of_vehicles, cost_G ,p,d)

12.0
[(0, 5), (1, 0), (2, 8), (3, 2), (5, 3), (8, 10), (10, 1), (0, 7), (4, 0), (6, 9), (7, 6), (9, 4)]
['0->5->3->2->8->10->1->0', '0->7->6->9->4->0']


### VRPB

In [18]:
def vrpb(number_of_clients, number_of_vehicles, cost_buffor, p_d):
    Q = 100
    m = number_of_vehicles

    L_B = list(range(1, number_of_clients + 1))
    SOURCE = 0

    L = [index for index, value in enumerate(p) if value == 0]
    L0 = L[:]
    L0.insert(0,0)
    B = [index for index, value in enumerate(d) if value == 0]

    B0 = B[:]
    B0.insert(0,0)
    Cu = L_B

    V = list(range(0, number_of_clients + 1))

    K = m
    D = p_d
    Kl = K
    Kb = K

    s = pulp.LpVariable.dicts('s', [(i, j) for i in V for j in V if i != j], cat = pulp.LpBinary)
    e = pulp.LpVariable.dicts('e', [(i, j) for i in L for j in B0 if i != j], cat = pulp.LpBinary)
    l = pulp.LpVariable.dicts('l', [(i, j) for i in V for j in V if i != j], 0, cat = "Continuous")

    model = pulp.LpProblem("vehicle_routing", pulp.LpMinimize)

    cost_function_model = pulp.lpSum(s[i,j] * cost_buffor[i][j] for i in V for j in V if i != j) + pulp.lpSum(e[i,j] * cost_buffor[i][j] for i in L for j in B0 if i != j)
    model += cost_function_model

    model += pulp.lpSum(s[i,j] for i in L0 for j in L if i != j) == len(L)

    for j in L:
      model += pulp.lpSum(l[i,j] for i in L0 if i != j) - pulp.lpSum(l[j,k] for k in L0 if k != j) == D[j]

    for j in L:
      model += pulp.lpSum(s[i,j] for i in L0 if i != j) == 1

    for j in L:
      model += pulp.lpSum(s[j,k] for k in L if k != j) + pulp.lpSum(e[j,k] for k in B0 if k != j)  ==  pulp.lpSum(s[(i,j)] for i in L0 if i != j)

    for j in L:
      for i in L0:
        if i != j:
          model += l[(i,j)] <= Q * s[(i,j)]

    fraaction7 = pulp.lpSum(D[j] for j in L)/Q
    model += pulp.lpSum(s[0,j] for j in L) >= max(float(fraaction7.value()),Kl)

    model += pulp.lpSum(s[i,j] for i in B for j in B0 if i != j) == len(B)

    for j in B:
      model += pulp.lpSum(l[i,j] for i in B if i != j) - pulp.lpSum(l[(j,k)] for k in B0 if k != j) == -D[j]

    for j in B:
      model += pulp.lpSum(s[j,i] for i in B0 if j != i) == 1

    for j in B:
      model += pulp.lpSum(s[k,j] for k in B if k != j) + pulp.lpSum(e[k,j] for k in L) == pulp.lpSum(s[(j,i)] for i in B0 if j != i)

    for j in B0:
      for i in B:
        if i != j:
          model += l[i,j] <= Q * s[i,j]

    fraaction13 = pulp.lpSum(D[j] for j in B)/Q
    model += max(int(fraaction13.value()),Kb) <= pulp.lpSum(s[i,0] for i in B)
    model += pulp.lpSum(s[(i,0)] for i in B) <= pulp.lpSum(s[0, j] for j in L)

    model += pulp.lpSum(s[i,0] for i in B) + pulp.lpSum(e[i, 0] for i in L) == pulp.lpSum(s[0,j] for j in L)

    for i in V:
      for j in V:
        if i != j:
          model += s[i,j] + s[j,i] <= 1

    model += pulp.lpSum(s[i,j] for i in B for j in L if i != j) == 0

    model += pulp.lpSum(s[0, j] for j in B) == 0

    model += pulp.lpSum(s[i,j] for i in L for j in B0 if i != j) == 0


    solver = pulp.apis.CPLEX_CMD()
    model.solve()
    print(pulp.value(model.objective))

    all_uesd_edges_road = []

    for i in V:
      for j in V:
        if i != j:
          if  s[i,j].varValue == 1:
            all_uesd_edges_road.append((i,j))

    for i in L:
      for j in B0:
        if i != j:
          if  e[i,j].varValue == 1:
            all_uesd_edges_road.append((i,j))

    print(all_uesd_edges_road)
    result = find_paths_vrb(all_uesd_edges_road, 2)

    print(result)

In [19]:
number_of_clients = 10
number_of_vehicles = 2

d = generate_demand_list(number_of_clients + 1, 3, 0.7 * 100 * number_of_vehicles)
p_d = replace_zeros_with_sum(d, 0.5 * 100 * number_of_vehicles)
p = [-1]

for iterator in range(1,len(d)):
  if d[iterator] == 0:
    p.append(p_d[iterator])
  else:
    p.append(0)

In [20]:
vrpb(number_of_clients, number_of_vehicles, cost_R, p_d)

12.0
[(0, 4), (0, 8), (1, 2), (2, 10), (3, 0), (4, 5), (6, 0), (7, 3), (8, 1), (10, 9), (5, 6), (9, 7)]
['0->4->5->6->0', '0->8->1->2->10->9->7->3->0']


In [21]:
vrpb(number_of_clients, number_of_vehicles, cost_G, p_d)

12.0
[(0, 4), (0, 8), (1, 2), (2, 10), (3, 0), (4, 5), (6, 0), (7, 3), (8, 1), (10, 9), (5, 6), (9, 7)]
['0->4->5->6->0', '0->8->1->2->10->9->7->3->0']
